# Adversarial Training

TJ Kim

11.29.21

#### Summary:
- Perform adversarial training in two different ways and analyze impact on inter-boundary distance of different variations of training
- Method 1: Train 100 rounds of regular FedEM/FedLocal (Same seed) --> 100 Rounds of adversarial Training
- Method 2: Train 200 rounds on benign model --> 200 rounds on adversarial model
- Examinations 1: Benign --> Adv trained 1 and 2 (Transferability and boundary distance)
- Examinations 2: Adv --> Adv trained 1 and 2 ("" "")

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


#### Import Libraries

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.Boundary_Transferer import *
from transfer_attacks.projected_gradient_descent import *

### Import FedEM Models

In [5]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_09_28_first_transfers/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 177.87it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:28<00:00,  2.78it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.159% |Test Loss: 2.292 | Test Acc: 12.248% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [6]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]
    


In [7]:
weights = np.load("weights/cifar/21_09_28_first_transfers/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

#print(weights)

# Set model weights
model_weights = []
num_models = 7

for i in range(num_models):
    model_weights += [weights[i]]
    
    
# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

In [8]:
clients[0].train_iterator.dataset.targets

tensor([5, 2, 6, 0, 1, 4, 6, 1, 2, 9, 6, 1, 0, 1, 1, 6, 6, 5, 5, 5, 1, 5, 0, 4,
        6, 9, 9, 2, 9, 5, 5, 1, 2, 5, 4, 2, 3, 6, 9, 6, 9, 6, 9, 6, 9, 1, 6, 2,
        9, 9, 1, 1, 2, 1, 1, 4, 9, 5, 9, 1, 5, 1, 6, 2, 9, 1, 5, 5, 9, 2, 6, 1,
        0, 0, 3, 4, 9, 1, 2, 4, 4, 4, 7, 6, 1, 1, 5, 4, 1, 2, 4, 4, 1, 6, 5, 4,
        0, 2, 0, 0, 4, 2, 6, 6, 9, 9, 6, 5, 0, 4, 1, 4, 5, 2, 6, 4, 0, 9, 1, 5,
        6, 0, 1, 6, 7, 8, 1, 4, 4, 4, 4, 6, 3, 6, 2, 9, 6, 0, 0, 5, 1, 0, 1, 9,
        9, 2, 2, 4, 5, 2, 2, 9, 1, 5, 0, 1, 9, 2, 1, 2, 6, 1, 3, 2, 4, 5, 9, 6,
        2, 0, 5, 4, 1, 2, 9, 0, 9, 0, 2, 5, 2, 9, 1, 4, 5, 0, 9, 9, 5, 0, 1, 5,
        0, 9, 5, 4, 2, 1, 6, 2, 4, 9, 6, 1, 1, 9, 1, 9, 5, 2, 6])

In [9]:
aggregator.clients[0].train_iterator.dataset.data[0].shape

torch.Size([32, 32, 3])

In [10]:
aggregator.clients[0].train_iterator.dataset.targets

tensor([5, 2, 6, 0, 1, 4, 6, 1, 2, 9, 6, 1, 0, 1, 1, 6, 6, 5, 5, 5, 1, 5, 0, 4,
        6, 9, 9, 2, 9, 5, 5, 1, 2, 5, 4, 2, 3, 6, 9, 6, 9, 6, 9, 6, 9, 1, 6, 2,
        9, 9, 1, 1, 2, 1, 1, 4, 9, 5, 9, 1, 5, 1, 6, 2, 9, 1, 5, 5, 9, 2, 6, 1,
        0, 0, 3, 4, 9, 1, 2, 4, 4, 4, 7, 6, 1, 1, 5, 4, 1, 2, 4, 4, 1, 6, 5, 4,
        0, 2, 0, 0, 4, 2, 6, 6, 9, 9, 6, 5, 0, 4, 1, 4, 5, 2, 6, 4, 0, 9, 1, 5,
        6, 0, 1, 6, 7, 8, 1, 4, 4, 4, 4, 6, 3, 6, 2, 9, 6, 0, 0, 5, 1, 0, 1, 9,
        9, 2, 2, 4, 5, 2, 2, 9, 1, 5, 0, 1, 9, 2, 1, 2, 6, 1, 3, 2, 4, 5, 9, 6,
        2, 0, 5, 4, 1, 2, 9, 0, 9, 0, 2, 5, 2, 9, 1, 4, 5, 0, 9, 9, 5, 0, 1, 5,
        0, 9, 5, 4, 2, 1, 6, 2, 4, 9, 6, 1, 1, 9, 1, 9, 5, 2, 6])

In [11]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

In [12]:
# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

In [22]:
img1 = clients[0].train_iterator.dataset.data[0]
img = Image.fromarray(img1.numpy())
tt = ToTensor()
tt(img)

tensor([[[0.0824, 0.0667, 0.0745,  ..., 0.0510, 0.0431, 0.0627],
         [0.0863, 0.0706, 0.0784,  ..., 0.0471, 0.0392, 0.0549],
         [0.0980, 0.0863, 0.0863,  ..., 0.0471, 0.0431, 0.0549],
         ...,
         [0.1843, 0.1686, 0.2235,  ..., 0.1176, 0.1176, 0.1059],
         [0.2000, 0.1922, 0.3255,  ..., 0.1216, 0.1216, 0.1059],
         [0.1686, 0.2118, 0.4745,  ..., 0.1294, 0.1216, 0.1020]],

        [[0.5647, 0.5686, 0.5882,  ..., 0.5765, 0.5765, 0.5804],
         [0.5647, 0.5647, 0.5804,  ..., 0.5686, 0.5686, 0.5686],
         [0.5686, 0.5765, 0.5843,  ..., 0.5686, 0.5725, 0.5725],
         ...,
         [0.6196, 0.5569, 0.5451,  ..., 0.5569, 0.6000, 0.6039],
         [0.6431, 0.5961, 0.6039,  ..., 0.5412, 0.5961, 0.6078],
         [0.6588, 0.6431, 0.6980,  ..., 0.5216, 0.5804, 0.5961]],

        [[0.7294, 0.7255, 0.7412,  ..., 0.7294, 0.7255, 0.7373],
         [0.7216, 0.7176, 0.7333,  ..., 0.7216, 0.7176, 0.7216],
         [0.7255, 0.7255, 0.7333,  ..., 0.7216, 0.7216, 0.

In [14]:

forward = Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
normed = clients[0].train_iterator.dataset.transform(img)

torch.reshape(img1,[3,32,32])
img1

tensor([[[ 21, 144, 186],
         [ 17, 145, 185],
         [ 19, 150, 189],
         ...,
         [ 13, 147, 186],
         [ 11, 147, 185],
         [ 16, 148, 188]],

        [[ 22, 144, 184],
         [ 18, 144, 183],
         [ 20, 148, 187],
         ...,
         [ 12, 145, 184],
         [ 10, 145, 183],
         [ 14, 145, 184]],

        [[ 25, 145, 185],
         [ 22, 147, 185],
         [ 22, 149, 187],
         ...,
         [ 12, 145, 184],
         [ 11, 146, 184],
         [ 14, 146, 185]],

        ...,

        [[ 47, 158, 192],
         [ 43, 142, 169],
         [ 57, 139, 157],
         ...,
         [ 30, 142, 173],
         [ 30, 153, 187],
         [ 27, 154, 189]],

        [[ 51, 164, 205],
         [ 49, 152, 180],
         [ 83, 154, 165],
         ...,
         [ 31, 138, 170],
         [ 31, 152, 185],
         [ 27, 155, 188]],

        [[ 43, 168, 211],
         [ 54, 164, 192],
         [121, 178, 185],
         ...,
         [ 33, 133, 166],
        

In [17]:
mean = torch.tensor([0.4914, 0.4822, 0.4465])
std = torch.tensor([0.2023, 0.1994, 0.201])

unnormalize = Normalize((-mean / std).tolist(), (1.0 / std).tolist())
a = unnormalize(normed)
a = a.transpose(0,1)
a = a.transpose(1,2)
a = a * 255
b = a.clone().detach().requires_grad_(True).type(torch.uint8)

In [18]:
b

tensor([[[ 20, 144, 186],
         [ 17, 145, 185],
         [ 18, 150, 189],
         ...,
         [ 12, 147, 186],
         [ 10, 147, 185],
         [ 15, 148, 188]],

        [[ 21, 144, 184],
         [ 17, 144, 183],
         [ 19, 148, 187],
         ...,
         [ 11, 145, 184],
         [  9, 145, 183],
         [ 13, 145, 184]],

        [[ 24, 145, 185],
         [ 21, 147, 185],
         [ 21, 149, 187],
         ...,
         [ 11, 145, 184],
         [ 10, 146, 184],
         [ 13, 146, 185]],

        ...,

        [[ 46, 158, 192],
         [ 42, 142, 169],
         [ 56, 139, 157],
         ...,
         [ 29, 142, 173],
         [ 29, 153, 187],
         [ 26, 154, 189]],

        [[ 50, 164, 205],
         [ 48, 152, 180],
         [ 82, 154, 165],
         ...,
         [ 30, 138, 170],
         [ 30, 152, 185],
         [ 26, 155, 188]],

        [[ 42, 168, 211],
         [ 53, 164, 192],
         [120, 178, 185],
         ...,
         [ 32, 133, 166],
        

In [26]:
def unnormalize_cifar10(normed):

    mean = torch.tensor([0.4914, 0.4822, 0.4465])
    std = torch.tensor([0.2023, 0.1994, 0.201])

    unnormalize = Normalize((-mean / std).tolist(), (1.0 / std).tolist())
    a = unnormalize(normed)
    a = a.transpose(0,1)
    a = a.transpose(1,2)
    a = a * 255
    b = a.clone().detach().requires_grad_(True).type(torch.uint8)
    
    return b

In [25]:
dataloader.x_data.shape

torch.Size([47965, 3, 32, 32])

In [29]:
data_xn = []
data_yn = []
rr = 1000

for i in range(rr):
    x = data_x[i]
    x_new = unnormalize_cifar10(x)
    
    data_xn.append(x_new)

data_xn = torch.stack(data_xn)


In [38]:
dataloader.y_data[0:10]

tensor([5, 2, 6, 0, 1, 4, 6, 1, 2, 9])

In [40]:
clients[0].train_iterator.dataset.data

tensor([[[[ 21, 144, 186],
          [ 17, 145, 185],
          [ 19, 150, 189],
          ...,
          [ 13, 147, 186],
          [ 11, 147, 185],
          [ 16, 148, 188]],

         [[ 22, 144, 184],
          [ 18, 144, 183],
          [ 20, 148, 187],
          ...,
          [ 12, 145, 184],
          [ 10, 145, 183],
          [ 14, 145, 184]],

         [[ 25, 145, 185],
          [ 22, 147, 185],
          [ 22, 149, 187],
          ...,
          [ 12, 145, 184],
          [ 11, 146, 184],
          [ 14, 146, 185]],

         ...,

         [[ 47, 158, 192],
          [ 43, 142, 169],
          [ 57, 139, 157],
          ...,
          [ 30, 142, 173],
          [ 30, 153, 187],
          [ 27, 154, 189]],

         [[ 51, 164, 205],
          [ 49, 152, 180],
          [ 83, 154, 165],
          ...,
          [ 31, 138, 170],
          [ 31, 152, 185],
          [ 27, 155, 188]],

         [[ 43, 168, 211],
          [ 54, 164, 192],
          [121, 178, 185],
         